# Analisis de propeidades estaticas
- Energía cinética media -> Temperatura media
- Fuerza cuadratica media
- Presión
- ...

## Importación de librerías y lectura de datos

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
import os
os.getcwd()

'/Users/xaime/Desktop/Cuarto/Simulacion/Programas/Proyectos/Notebooks'

In [4]:
from pathlib import Path
#codigo ="110-160"

ruta_base = Path("..") / "resultados" / "dinamica"

ruta=ruta_base
print(ruta.exists())  # True o False
print(ruta.parent)    # Ruta al directorio padre
print(ruta.name)      # archivo.txt
print(ruta.stem)      # archivo

True
../resultados
dinamica
dinamica


In [5]:
#pd.read_csv(ruta, sep=r'\s+')

In [6]:
codigos=["110-160","160-210","110-160","210-260","260-310","310-360","360-410","410-460","460-510","510-560"] #Tiempo de inicio de simulacion-Tiempo de fin de simulacion

DFS_vx=[]
DFS_vy=[]
DFS_vz=[]

DATOS=[]

for codigo in codigos:
    DFS_vx.append(pd.read_csv(ruta_base / f"Simulacion_{codigo}"/"velocidad_x.txt", sep=r"\s+", header=None))
    DFS_vy.append(pd.read_csv(ruta_base/ f"Simulacion_{codigo}"/"velocidad_y.txt", sep=r"\s+", header=None))
    DFS_vz.append(pd.read_csv(ruta_base / f"Simulacion_{codigo}"/"velocidad_z.txt", sep=r"\s+", header=None))
    DATOS.append(pd.read_csv(ruta_base/ f"Simulacion_{codigo}"/"gname2.txt",skiprows=1, sep=r"\s+", header=None, names=['t','Ec','Ep','E_tot','W','T','P_corr','F2']))

In [7]:
tiempos=[df.t.to_numpy() for df in DATOS]

tiempos[0]

array([1.000e-02, 2.000e-02, 3.000e-02, ..., 4.998e+01, 4.999e+01,
       5.000e+01], shape=(5000,))

### Formato de datos importados

- En las columnas tenemos las particulas de la caja etiquetadas del 0 al 499
- En las filas tenemos instantes sucesivos de tiempo grabados de nuestra simulacion.

In [14]:
DATOS[-1].iloc[:,1:].agg(["mean","std"])
#DATOS[-1].min()

,Ec,Ep,E_tot,W,T,P_corr,F2
mean,1024.125861,-1689.140024,-665.014171,-1283.983673,1.365501,0.221247,564.653986
std,17.820872,17.821100,0.022239,334.197115,0.023761,0.104360,80.949327


In [15]:
F2s_mean=[]
F2s_std=[]

Ecs_mean=[]
Ecs_std=[]

Eps_mean=[]
Eps_std=[]

Ws_mean=[]
Ws_std=[]

Ps_mean=[]
Ps_std=[]


for df in DATOS:
    F2s_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["mean","F2"])
    F2s_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["std","F2"])

    Ecs_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["mean","Ec"])
    Ecs_std.append(df.iloc[:,1:].agg(["mean","std"]).loc["std","Ec"])

    Eps_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["mean","Ep"])
    Eps_std.append(df.iloc[:,1:].agg(["mean","std"]).loc["std","Ep"])

    Ws_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["mean","W"])
    Ws_std.append(df.iloc[:,1:].agg(["mean","std"]).loc["std","W"])

    Ps_mean.append(df.iloc[:,1:].agg(["mean","std"]).loc["mean","P_corr"])
    Ps_std.append(df.iloc[:,1:].agg(["mean","std"]).loc["std","P_corr"])


F2s_mean=np.array(F2s_mean)
F2s_std=np.array(F2s_mean)

Ecs_mean=np.array(Ecs_mean)
Ecs_std=np.array(Ecs_std)

Eps_mean=np.array(Eps_mean)
Eps_std=np.array(Eps_std)

Ws_mean=np.array(Ws_mean)
Ws_std=np.array(Ws_std)

Ps_mean=np.array(Ps_mean)
Ps_std=np.array(Ps_std)


"""
print("Ec=",)

print(F2s_mean)
print(F2s_std)
print(Ecs_mean)
print(Ecs_std)
print(Ps_mean)
print(Ps_std)

"""

# Media global = media de las medias
F2_mean = np.mean(F2s_mean)
Ec_mean = np.mean(Ecs_mean)
Ep_mean = np.mean(Eps_mean)
W_mean = np.mean(Ws_mean)
P_mean  = np.mean(Ps_mean)

# Error estándar de la media global --> ¿Deberia tener en cuenta las std de cada bloque simulado?
F2_err = np.std(F2s_mean, ddof=1) / np.sqrt(len(F2s_mean))
Ec_err = np.std(Ecs_mean, ddof=1) / np.sqrt(len(Ecs_mean))
Ep_err = np.std(Eps_mean, ddof=1) / np.sqrt(len(Eps_mean))
W_err = np.std(Ws_mean, ddof=1) / np.sqrt(len(Ws_mean))
P_err  = np.std(Ps_mean, ddof=1) / np.sqrt(len(Ps_mean))

print(f"<F²> = {F2_mean:.6f} ± {F2_err:.6f}")
print(f"<Ec> = {Ec_mean:.6f} ± {Ec_err:.6f}")
print(f"<Ep> = {Ep_mean:.6f} ± {Ep_err:.6f}")
print(f"<W> = {W_mean:.6f} ± {W_err:.6f}")
print("W/3V ES LA DERIVADA PRIMERA DEL POTENCIAL")
print("FALTAN LA DERIVADA SEGUNDA DEL POTENCIAL")

print(f"<P>  = {P_mean:.6f} ± {P_err:.6f}")

<F²> = 326.687683 ± 56.087657
<Ec> = 1023.630884 ± 0.694273
<Ep> = -1688.653544 ± 0.695153
<W> = -1247.969042 ± 5.551650
W/3V ES LA DERIVADA PRIMERA DEL POTENCIAL
FALTAN LA DERIVADA SEGUNDA DEL POTENCIAL
<P>  = 0.232922 ± 0.002128


# Propiedades estaticas

$ C_v = \frac{3}{2} + \frac{1}{T} \left[ <\phi_V^2>-<\phi_V>^2\right]$

$\frac{1}{k_T}= \frac{NT}{V} + V<\phi_VV>-\frac{V}{T} \left[  <\phi_V^2>-<\phi_V>^2\right]$

$\gamma = \frac{V}{C_V}\left[\frac{N k}{V} +\frac{1}{k T^2}\left(\langle \phi \rangle \langle \phi_V \rangle - \langle \phi \phi_V\rangle\right)\right]$


In [17]:
def Cv(T,W2_mean,W_mean2):
    return 3/2

def Kt_1(N,T,V,W2_mean,W_mean2):
    return 32

def gamma():
    return